# BANK MARKETING

Name : SAYANA RAHUL

Organization : Entri Elevate

Date : [Evaluation Date]


Introduction:The Bank Marketing dataset contains information from direct phone call marketing campaigns conducted by a Portuguese banking institution. The dataset includes various client attributes, such as age, job, marital status, and education, as well as details about the marketing campaign itself. The primary objective is to predict whether a client will subscribe to a term deposit based on these attributes. This prediction aids in enhancing the effectiveness of future marketing strategies.

Objective:The objective is to Develop a machine learning model to accurately predict client subscription to term deposits, thereby enhancing the efficiency of future marketing campaigns.

Data Description: The Bank Marketing dataset contains data from direct phone call marketing campaigns conducted by a Portuguese banking institution. It includes 45,211 instances and 16 features, capturing client demographics, contact details, and campaign specifics.

Source: UCI Machine Learning Repository

Features: 1 - age (numeric) 2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services") 3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed) 4 - education (categorical: "unknown","secondary","primary","tertiary") 5 - default: has credit in default? (binary: "yes","no") 6 - balance: average yearly balance, in euros (numeric) 7 - housing: has housing loan? (binary: "yes","no") 8 - loan: has personal loan? (binary: "yes","no") related with the last contact of the current campaign: 9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 10 - day: last contact day of the month (numeric) 11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec") 12 - duration: last contact duration, in seconds (numeric) other attributes: 13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact) 14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted) 15 - previous: number of contacts performed before this campaign and for this client (numeric) 16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

Output variable (desired target): 17 - y - has the client subscribed a term deposit? (binary: "yes","no")


# Data Collection

In [3]:
#Import Necessary Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy.stats import zscore, skew 
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler


In [4]:
# Load the dataset into a DataFrame
data = pd.read_csv('bank12.csv',delimiter=';')


In [5]:
data.head()

,"age;""job"";""marital"";""education"";""default"";""balance"";""housing"";""loan"";""contact"";""day"";""month"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""y"""
0,"30;""unemployed"";""married"";""primary"";""no"";1787;..."
1,"33;""services"";""married"";""secondary"";""no"";4789;..."
2,"35;""management"";""single"";""tertiary"";""no"";1350;..."
3,"30;""management"";""married"";""tertiary"";""no"";1476..."
4,"59;""blue-collar"";""married"";""secondary"";""no"";0;..."


In [6]:
bank_df=pd.DataFrame(data)


In [7]:
bank_df

,"age;""job"";""marital"";""education"";""default"";""balance"";""housing"";""loan"";""contact"";""day"";""month"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""y"""
0,"30;""unemployed"";""married"";""primary"";""no"";1787;..."
1,"33;""services"";""married"";""secondary"";""no"";4789;..."
2,"35;""management"";""single"";""tertiary"";""no"";1350;..."
3,"30;""management"";""married"";""tertiary"";""no"";1476..."
4,"59;""blue-collar"";""married"";""secondary"";""no"";0;..."
...,...
4516,"33;""services"";""married"";""secondary"";""no"";-333;..."
4517,"57;""self-employed"";""married"";""tertiary"";""yes"";..."
4518,"57;""technician"";""married"";""secondary"";""no"";295..."
4519,"28;""blue-collar"";""married"";""secondary"";""no"";11..."


In [8]:
# Display the summary of the dataset
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 1 columns):
 #   Column                                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                                --------------  ----- 
 0   age;"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"  4521 non-null   object
dtypes: object(1)
memory usage: 35.4+ KB
None


In [9]:
# Display statistical summary of numerical columns
print(data.describe())


       age;"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
count                                                4521                                                                                                  
unique                                               4521                                                                                                  
top     30;"unemployed";"married";"primary";"no";1787;...                                                                                                  
freq                                                    1                                                                                                  


# Data Processing

In [10]:
# Check for missing values
print("Missing values:\n", data.isnull().sum())


Missing values:
 age;"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"    0
dtype: int64


In [15]:
#impute missing numerical values with mean
num_imputer = SimpleImputer(strategy='mean')
data[data.select_dtypes(include=['float64', 'int64']).columns] = num_imputer.fit_transform(data.select_dtypes(include=['float64', 'int64']))


ValueError: at least one array or dtype is required

In [16]:
print("Missing values after handling:\n", data.isnull().sum())
print("Missing values have been successfully handled.")


Missing values after handling:
 age;"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"    0
dtype: int64
Missing values have been successfully handled.


In [18]:
#Check for and Remove Outliers
# Box plot to visualize outliers
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    plt.figure(figsize=(3, 2))
    sns.boxplot(data[col])
    plt.title(f"Boxplot of {col}")
    plt.show()
    

In [21]:
# Remove outliers using IQR
for col in numerical_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    

In [23]:
#Boxplot After Removing Outliers
for col in numerical_cols:
    plt.figure(figsize=(3, 2))
    sns.boxplot(data[col])
    plt.title(f"Boxplot of {col} (After Removing Outliers)")
    plt.show()
    


In [24]:
# Address Skewed Data
# Check skewness
skewness = data[numerical_cols].apply(lambda x: skew(x.dropna()))
print("Skewness before transformation:\n", skewness)


Skewness before transformation:
 Series([], dtype: float64)


C:\Users\srahu\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1193: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=True)
C:\Users\srahu\anaconda3\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\srahu\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\srahu\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [26]:
# Apply PowerTransformer to normalize skewed data
pt = PowerTransformer(method='yeo-johnson')
data[numerical_cols] = pt.fit_transform(data[numerical_cols])


ValueError: at least one array or dtype is required

In [27]:
# Validate transformations
skewness_after = data[numerical_cols].apply(lambda x: skew(x))
print("Skewness after transformation:\n", skewness_after)


Skewness after transformation:
 Series([], dtype: float64)


C:\Users\srahu\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1193: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=True)
C:\Users\srahu\anaconda3\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\srahu\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\srahu\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Exploratory Data Analysis (EDA)

In [30]:
# 1.Histogram: Data distribution for numerical columns
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    plt.figure(figsize=(3, 2))
    plt.hist(data[col], bins=20, edgecolor='k', alpha=0.7)
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()
    

In [32]:
# 2.Boxplot: Identifying outliers
for col in numerical_cols:
    plt.figure(figsize=(3, 2))
    sns.boxplot(data[col])
    plt.title(f"Boxplot of {col}")
    plt.show()
    

In [34]:
# 3.Pair Plot: Relationships between numerical features
sns.pairplot(data)
plt.show()

ValueError: No variables found for grid columns.

In [ ]:
# 4 Pie Diagram: Proportions in categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, counterclock=False)
    plt.title(f"Pie Chart of {col}")
    plt.ylabel("")  # Hide y-axis label
    plt.show()

In [ ]:
# 6 Count Plot: Count visualization for categorical features
for col in categorical_cols:
    plt.figure(figsize=(3, 2))
    sns.countplot(data=data, x=col, order=data[col].value_counts().index)
    plt.title(f"Count Plot of {col}")
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# 7 Line Plot: Trend visualization for numerical columns
for col in numerical_cols:
    plt.figure(figsize=(3, 2))
    plt.plot(data.index, data[col])
    plt.title(f"Line Plot of {col}")
    plt.xlabel("Index")
    plt.ylabel(col)
    plt.show()

In [ ]:
# 8 Kernel Density Estimation (KDE): Data density
for col in numerical_cols:
    plt.figure(figsize=(3, 2))
    sns.kdeplot(data[col], fill=True, alpha=0.6)
    plt.title(f"KDE of {col}")
    plt.xlabel(col)
    plt.ylabel("Density")
    plt.show()

# Feature Engineering

In [ ]:
# Identify categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns

# One-hot encoding for categorical features
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Display the transformed dataset
print("\nData After One-Hot Encoding:")
print(data_encoded.head())


# Feature Extraction

In [ ]:
target_column = 'y' 
X = data.drop(target_column, axis=1)  
y = data[target_column] 

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Forest for feature importance
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)

# Select top-k features based on importance
k = 5  # Number of features to select
selected_features = feature_importances.nlargest(k).index
X_selected = X[selected_features]

# Scaling the selected features
# Initialize the scalers
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Apply Standard Scaling
X_standardized = standard_scaler.fit_transform(X_selected)

# Apply Min-Max Scaling
X_normalized = minmax_scaler.fit_transform(X_selected)

# Convert the scaled data to DataFrames for better readability
X_standardized_df = pd.DataFrame(X_standardized, columns=selected_features)
X_normalized_df = pd.DataFrame(X_normalized, columns=selected_features)

# Display the scaled DataFrames
print("Standardized Features (Z-score Scaling):")
print(X_standardized_df)

print("\nNormalized Features (Min-Max Scaling):")
print(X_normalized_df)


# Split Data Into Training And Testing Sets

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shape of training and testing sets
print(f"\nTraining Set Shape: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Testing Set Shape: X_test: {X_test.shape}, y_test: {y_test.shape}")
